In [1]:
# Import the dependencies.
import numpy as np
import pandas as pd
from citipy import citipy # Use the citipy module to determine city based on latitude and longitude.
import requests
from datetime import datetime
import time

# Import the API key.
# Import API key
import sys
import os
sys.path.append(os.path.abspath('..'))
from config import weather_api_key

**Get the latitude and longitude combinations.**

In [2]:
# Create a set of random latitude and longitude combinations.
objective_cities = 50
# size = int(round(objective_cities*2.5,0))
size = 2000

lats = np.random.uniform(low=-90.000, high=90.000, size=size)
lngs = np.random.uniform(low=-180.000, high=180.000, size=size)

# Pack the latitudes and longitudes as pairs
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

**Get the nearest city using the $citipy$ module.**

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

769

**Perform an API call with the OpenWeatherMap and retrieve the following information from the API call:**

    - Latitude and longitude
    - Maximum temperature
    - Percent humidity
    - Percent cloudiness
    - Wind speed
    - Weather description (for example, clouds, fog, light rain, clear sky)

In [4]:
# URL and Parameters

# Base URL
base_url =  "http://api.openweathermap.org/data/2.5/weather?" + "appid=" + weather_api_key + "&units=metric"

---

**Example:**

In [5]:
ex_city = cities[3]
city_url = base_url + "&q=" + ex_city.replace(" ","+")

In [6]:
city_weather = requests.get(city_url).json()
city_weather

{'coord': {'lon': -70.9167, 'lat': -53.15},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 8.95,
  'feels_like': 4.79,
  'temp_min': 8.95,
  'temp_max': 8.95,
  'pressure': 1012,
  'humidity': 49,
  'sea_level': 1012,
  'grnd_level': 1009},
 'visibility': 10000,
 'wind': {'speed': 10.15, 'deg': 298, 'gust': 18.94},
 'clouds': {'all': 53},
 'dt': 1636610463,
 'sys': {'type': 2,
  'id': 47745,
  'country': 'CL',
  'sunrise': 1636620099,
  'sunset': 1636676036},
 'timezone': -10800,
 'id': 3874787,
 'name': 'Punta Arenas',
 'cod': 200}

In [7]:
print(f"City: {ex_city.title()}\n"
      f"Country Code: {city_weather['sys']['country']}\n"
      f"Latitude and Longitude: ({city_weather['coord']['lat']},{city_weather['coord']['lon']})\n"
      f"Maximum Temperature: {city_weather['main']['temp_max']} C°\n"
      f"Percent Humidity: {city_weather['main']['humidity']}%\n"
      f"Percent Cloudiness: {city_weather['clouds']['all']}%\n"
      f"Wind Speed: {city_weather['wind']['speed']} meter/sec\n"
      f"Weather Description: {city_weather['weather'][0]['description'].title()}\n"
      f"City Date:{datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')}"
     )

City: Punta Arenas
Country Code: CL
Latitude and Longitude: (-53.15,-70.9167)
Maximum Temperature: 8.95 C°
Percent Humidity: 49%
Percent Cloudiness: 53%
Wind Speed: 10.15 meter/sec
Weather Description: Broken Clouds
City Date:2021-11-11 06:01:03


---

In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------\n\n")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        print(f"Processing Set {set_count}  -------------")

    # Create endpoint URL with each city.
    city_url = base_url + "&q=" + city.replace(" ","+")

    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.        
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_description = city_weather['weather'][0]['description']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S') # Date to ISO standard.
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description.title(),
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print(f"{city.title()} City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("\n\n-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------


Kamenskoye City not found. Skipping...
Taolanaro City not found. Skipping...
Mys Shmidta City not found. Skipping...
Illoqqortoormiut City not found. Skipping...
Tapaua City not found. Skipping...
Barentsburg City not found. Skipping...
Processing Set 2  -------------
Inveraray City not found. Skipping...
Xiangdong City not found. Skipping...
Amderma City not found. Skipping...
Belushya Guba City not found. Skipping...
Processing Set 3  -------------
Grand River South East City not found. Skipping...
Jiddah City not found. Skipping...
Processing Set 4  -------------
Karauzyak City not found. Skipping...
Sentyabrskiy City not found. Skipping...
Processing Set 5  -------------
Nizhneyansk City not found. Skipping...
Bolungarvik City not found. Skipping...
Mochitlan City not found. Skipping...
Palabuhanratu City not found. Skipping...
Vaitupu City not found. Skipping...
Processing Set 6  -------------
Samalaeulu City not found.

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df =  city_data_df[["City","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description","Date"]]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,Chokurdakh,RU,70.6333,147.9167,-19.00,94,98,3.17,Overcast Clouds,2021-11-11 06:05:41
1,Cabo San Lucas,MX,22.8909,-109.9124,25.28,71,0,1.33,Clear Sky,2021-11-11 06:02:39
2,Praia,CV,14.9215,-23.5087,23.30,83,0,3.09,Clear Sky,2021-11-11 06:05:42
3,Punta Arenas,CL,-53.1500,-70.9167,8.95,49,53,10.15,Broken Clouds,2021-11-11 06:01:03
4,Ushuaia,AR,-54.8000,-68.3000,4.81,93,0,1.03,Clear Sky,2021-11-11 06:05:43
5,Taraz,KZ,42.9000,71.3667,10.23,40,0,1.00,Clear Sky,2021-11-11 06:05:44
6,Bluff,NZ,-46.6000,168.3333,15.75,78,96,6.95,Overcast Clouds,2021-11-11 06:01:12
7,Mezen,RU,65.8522,44.2400,0.58,98,100,4.03,Overcast Clouds,2021-11-11 06:05:45
8,Avarua,CK,-21.2078,-159.7750,24.03,78,75,6.69,Broken Clouds,2021-11-11 06:05:45
9,Harper,LR,4.3750,-7.7169,25.40,87,87,1.88,Light Rain,2021-11-11 06:05:46


In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
len(city_data_df)

714